*Теоретический материал:* https://youtu.be/kSFrLjBWbd4?t=153

# ONNX + Keras (Restart Runtime)

*Разбор данного раздела:* https://youtu.be/kSFrLjBWbd4?t=8214

In [ ]:
!pip install keras2onnx
!pip install onnxruntime
!pip install tensorflow==2.3

     |████████████████████████████████| 102kB 5.4MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 7.7MB 12.3MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111006 sha256=5f20166f315b0a4371d952ae116454299d49bbe3b62e03eb58556c760c6bf4c7
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire
     |████████████████████████████████| 4.1MB 13.3MB/s 
     |████████████████████████████████| 320.4MB 53kB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 460kB 44.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.4
  

In [ ]:
# Если вы загружаете это в колабе
# база данных фото цветов
!curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
# разархивировать в текущую директорию
!tar xzf flower_photos.tgz
# создаем папку с образцами
!mkdir flower_photos/sample 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0   321M      0 --:--:-- --:--:-- --:--:--  321M
mkdir: cannot create directory ‘flower_photos/sample’: File exists


In [ ]:
# Эти три фото поместим в папку 
!cp flower_photos/daisy/5547758_eea9edfd54_n.jpg flower_photos/sample/
!cp flower_photos/daisy/10140303196_b88d3d6cec.jpg flower_photos/sample/
!cp flower_photos/tulips/100930342_92e8746431_n.jpg flower_photos/sample/

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import keras2onnx
import onnxruntime
from keras.applications.resnet50 import ResNet50

# предобработка изображения
img_path = 'flower_photos/sample/10140303196_b88d3d6cec.jpg'   
img_size = 224 # желаемый размер
img = image.load_img(img_path, target_size=(img_size, img_size)) # используем стандартную керасовскую функцию для предобработки
x = image.img_to_array(img) # преобразуем в массив
x = np.expand_dims(x, axis=0) # докидываем еще измерение, чтоб был 4D массив 
x = preprocess_input(x) # функция, нормализующая batch под нужды конкретно этой сетки

# загрузим предобученную сеть
model = ResNet50(include_top=True, weights='imagenet')

# конвертируем в onnx
onnx_model = keras2onnx.convert_keras(model, model.name)

102973440/102967424 [==============================] - 1s 0us/step


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 458 -> 127


In [ ]:
# предсказание от onnxruntime
content = onnx_model.SerializeToString() # сериализуем

sess = onnxruntime.InferenceSession(content) # инференс этого представления в сессию

x = x if isinstance(x, list) else [x] # если принадлежит классу список

In [ ]:
print(len(x))
print(x[0].shape)

1
(1, 224, 224, 3)


In [ ]:
sess.get_inputs()[0].name

'input_1'

In [ ]:
# на вход сессии всегда подается словарь "название":значение
feed = dict([(inputs.name, x[n]) for n, inputs in enumerate(sess.get_inputs())]) # sess.get_inputs() - получить описание того, что будет подаваться на вход

In [ ]:
feed

{'input_1': array([[[[ 116.061   ,   95.221   ,   88.32    ],
          [ 115.061   ,   96.221   ,   82.32    ],
          [ 113.061   ,   93.221   ,   83.32    ],
          ...,
          [ -93.939   , -110.779   , -118.68    ],
          [ -91.939   , -109.779   , -114.68    ],
          [ -92.939   , -110.779   , -116.68    ]],
 
         [[ 121.061   ,  101.221   ,   91.32    ],
          [ 127.061   ,  107.221   ,   97.32    ],
          [ 123.061   ,  102.221   ,   95.32    ],
          ...,
          [ -89.939   , -107.779   , -113.68    ],
          [ -85.939   , -103.779   , -109.68    ],
          [ -88.939   , -105.779   , -113.68    ]],
 
         [[ 121.061   ,  100.221   ,   93.32    ],
          [ 122.061   ,  101.221   ,   94.32    ],
          [ 129.061   ,  109.221   ,   99.32    ],
          ...,
          [ -92.939   , -110.779   , -116.68    ],
          [ -93.939   , -111.779   , -119.68    ],
          [ -91.939   , -108.779   , -116.68    ]],
 
         ...,
 
 

In [ ]:
pred_onnx = sess.run(None, feed) # None, т.к. нет outputs(указание какого-то конкретного лейбла), есть только inputs

In [ ]:
print(np.array(pred_onnx).shape)

(1, 1, 1000)


In [ ]:
np.array(pred_onnx)[0,0,:20]

array([2.4703264e-10, 3.2971961e-08, 6.1456146e-10, 1.0135802e-10,
       8.6856206e-10, 3.0754048e-09, 5.4214272e-10, 1.4601862e-08,
       2.1090092e-09, 2.7297657e-09, 2.3581082e-08, 2.0554364e-07,
       3.4769780e-08, 2.6511168e-07, 1.1332778e-07, 5.0114792e-09,
       3.0195373e-08, 3.9996340e-08, 4.6655781e-08, 3.9242295e-07],
      dtype=float32)

In [ ]:
np.argmax(np.array(pred_onnx))

985

###Пример с Sklearn (из документации от Microsoft)

*Разбор данного раздела:* https://youtu.be/kSFrLjBWbd4?t=8610


In [ ]:
pip install skl2onnx

     |████████████████████████████████| 194kB 14.0MB/s 


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import numpy
import onnxruntime

iris = load_iris() # загружаем стандартный датасет ирисов
X, y = iris.data, iris.target # вытаскиваем x и y  
X_train, X_test, y_train, y_test = train_test_split(X, y) # делим на обучающую и тренировочную выборку

clr = LogisticRegression() # объект логистической регресии
clr.fit(X_train, y_train) # скармливаем
print(clr) # посмотрим, что собой представляет этот объект

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
X_train[:3]

array([[5. , 3.3, 1.4, 0.2],
       [6.1, 2.9, 4.7, 1.4],
       [6.8, 2.8, 4.8, 1.4]])

In [ ]:
y_train[:5]

array([0, 1, 1, 2, 1])

In [ ]:
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type) # преобразуем в onnx формат для дальнейшего сохранения

with open("logreg_iris.onnx", "wb") as f: # сериализуем
    f.write(onx.SerializeToString())

In [ ]:
sess = onnxruntime.InferenceSession("logreg_iris.onnx") # стартуем ONNX Runtime сессию

input_name = sess.get_inputs()[0].name # вытаскиваем вшитое имя для инпутов
pred_onx = sess.run(None, {input_name: X_test.astype(numpy.float32)}) # прогоняем данные проверочной выборки через сессию
print(pred_onx) # предсказание от ONNX runtime 

[array([0, 1, 0, 2, 2, 2, 0, 1, 0, 1, 0, 0, 1, 0, 2, 0, 2, 2, 0, 2, 1, 0,
       0, 2, 1, 1, 1, 2, 1, 0, 2, 0, 1, 0, 2, 2, 0, 0], dtype=int64), [{0: 0.9745892286300659, 1: 0.025410817936062813, 2: 2.9038748294851757e-08}, {0: 0.01212165504693985, 1: 0.8902363777160645, 2: 0.0976419746875763}, {0: 0.9686844944953918, 1: 0.03131549060344696, 2: 4.168359168943425e-08}, {0: 8.642266038805246e-06, 1: 0.028435954824090004, 2: 0.9715553522109985}, {0: 1.0219214807705157e-08, 1: 0.0018424374284222722, 2: 0.9981575608253479}, {0: 8.481775637392275e-08, 1: 0.0073696612380445, 2: 0.9926302433013916}, {0: 0.9643263220787048, 1: 0.03567364439368248, 2: 5.586887752428993e-08}, {0: 0.0023713302798569202, 1: 0.5166107416152954, 2: 0.48101794719696045}, {0: 0.9661330580711365, 1: 0.033866964280605316, 2: 3.889915234367436e-08}, {0: 0.0016109129646793008, 1: 0.5400652885437012, 2: 0.45832377672195435}, {0: 0.9734315872192383, 1: 0.02656838297843933, 2: 6.243607941769369e-08}, {0: 0.9613251090049744, 1: 

In [ ]:
y_test # правильный ответ

array([0, 1, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 2, 1, 0, 2, 1, 0,
       0, 2, 1, 1, 1, 2, 1, 0, 2, 0, 2, 0, 2, 2, 0, 0])

In [ ]:
sess.get_outputs()[1].name

'output_probability'

In [ ]:
sess.get_outputs()[0].name

'output_label'